###1.Модифицировать код функции get_user_embedding таким образом, чтобы считалось не среднее (как в примере np.mean), а медиана.

###2.Применить такое преобразование к данным, обучить модель прогнозирования оттока и посчитать метрики качества и сохранить их: roc auc, precision/recall/f_score (для 3 последних - подобрать оптимальный порог с помощью precision_recall_curve, как это делалось на уроке)

###3.Повторить п.2, но используя уже не медиану, а max (опциональное, если очень хочется) Воспользовавшись полученными знаниями из п.1, повторить пункт 2, но уже взвешивая новости по tfidf (подсказка: нужно получить веса-коэффициенты для каждого документа. Не все документы одинаково информативны и несут какой-то положительный сигнал). Подсказка 2 - нужен именно idf, как вес.

###4.Сформировать на выходе единую таблицу, сравнивающую качество 3 разных метода получения эмбедингов пользователей: mean, median, max, idf_mean по метрикам roc_auc, precision, recall, f_score Сделать самостоятельные выводы и предположения о том, почему тот или ной способ оказался эффективнее остальных

In [14]:
import pandas as pd

In [15]:
news = pd.read_csv("articles.csv",
                   encoding='utf-8')
                    
print(news.shape)
news.head(3)

(27000, 2)


,doc_id,title
0,6,Заместитель председателяnправительства РФnСерг...
1,4896,Матч 1/16 финала Кубка России по футболу был п...
2,4897,Форвард «Авангарда» Томаш Заборский прокоммент...


Загрузим пользователей и списки последних прочитанных новостей

In [16]:
users = pd.read_csv("users_articles.csv")
users.head(3)

,uid,articles
0,u105138,"[293672, 293328, 293001, 293622, 293126, 1852]"
1,u108690,"[3405, 1739, 2972, 1158, 1599, 322665]"
2,u108339,"[1845, 2009, 2356, 1424, 2939, 323389]"


###Получаем векторные представления новостей

In [17]:
from gensim.test.utils import common_texts
from gensim.corpora.dictionary import Dictionary

In [18]:
!pip install razdel


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [19]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 KB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 56.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13721 sha256=82e40c9074f6f018279f598071f7ef320d405588550d5af562a71e71d614625c
  Stored in directory: /root/.cache/pip/wheels/70/4a/46/1309fc853b8d395e60bafaf1b6df7845bdd82c95fd59dd8d2b
Successfully built docopt


In [20]:
#предобработка текстов
import re
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from razdel import tokenize

import pymorphy2  

In [21]:
 import nltk
 nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [22]:
stopword_ru = stopwords.words('russian')
len(stopword_ru)

morph = pymorphy2.MorphAnalyzer()

In [23]:
with open('stopwords.txt') as f:
    additional_stopwords = [w.strip() for w in f.readlines() if w]
stopword_ru += additional_stopwords
len(stopword_ru)

776

In [24]:
def clean_text(text):
    '''
    очистка текста
       
    '''
    if not isinstance(text, str):
        text = str(text)
    
    text = text.lower()
    text = text.strip('\n').strip('\r').strip('\t')
    text = re.sub("-\s\r\n\|-\s\r\n|\r\n", '', str(text))

    text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)
    text = re.sub(r"\r\n\t|\n|\\s|\r\t|\\n", ' ', text)
    text = re.sub(r'[\xad]|[\s+]', ' ', text.strip())
    
   
    return text

cache = {}

def lemmatization(text):
    '''
    лемматизация
        [0] если зашел тип не `str` делаем его `str`
        [1] токенизация предложения через razdel
        [2] проверка есть ли в начале слова '-'
        [3] проверка токена с одного символа
        [4] проверка есть ли данное слово в кэше
        [5] лемматизация слова
        [6] проверка на стоп-слова

    на выходе лист отлемматизированых токенов
    '''

    # [0]
    if not isinstance(text, str):
        text = str(text)
    
    # [1]
    tokens = list(tokenize(text))
    words = [_.text for _ in tokens]

    words_lem = []
    for w in words:
        if w[0] == '-': # [2]
            w = w[1:]
        if len(w)>1: # [3]
            if w in cache: # [4]
                words_lem.append(cache[w])
            else: # [5]
                temp_cach = cache[w] = morph.parse(w)[0].normal_form
                words_lem.append(temp_cach)
    
    words_lem_without_stopwords=[i for i in words_lem if not i in stopword_ru] # [6]
    
    return words_lem_without_stopwords

In [25]:
%%time
#Запускаем очистку текста. Будет долго...
news['title'] = news['title'].apply(lambda x: clean_text(x), 1)

<ipython-input-24-2971e9a97abc>:13: FutureWarning: Possible nested set at position 39
  text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)


CPU times: user 40.8 s, sys: 1.56 s, total: 42.3 s
Wall time: 52.3 s


In [26]:
%%time
#Запускаем лемматизацию текста. Будет очень долго...
news['title'] = news['title'].apply(lambda x: lemmatization(x), 1)

CPU times: user 4min 7s, sys: 793 ms, total: 4min 8s
Wall time: 4min 22s


А теперь в 3 строчки обучим нашу модель

In [27]:
texts = [t for t in news['title'].values]

common_dictionary = Dictionary(texts)
common_corpus = [common_dictionary.doc2bow(text) for text in texts]

Запускаем обучение

In [28]:
from gensim.models import LdaModel

lda = LdaModel(common_corpus, num_topics=25, id2word=common_dictionary)

In [29]:
from gensim.test.utils import datapath
# Save model to disk.
temp_file = datapath("model.lda")
lda.save(temp_file)

# Load a potentially pretrained model from disk.
lda = LdaModel.load(temp_file)

Обучили модель. Теперь 2 вопроса:

1. как выглядят наши темы
2. как получить для документа вектор значений (вероятности принадлежности каждой теме)

In [30]:
# Create a new corpus, made of previously unseen documents.
other_texts = [t for t in news['title'].iloc[:3]]
other_corpus = [common_dictionary.doc2bow(text) for text in other_texts]

unseen_doc = other_corpus[2]
print(other_texts[2])
lda[unseen_doc] 

['форвард', 'авангард', 'томаш', 'заборский', 'прокомментировать', 'игра', 'команда', 'матч', 'чемпионат', 'кхл', 'против', 'атланта', 'nnnn', 'плохой', 'матч', 'нижний', 'новгород', 'против', 'торпедо', 'настраиваться', 'первый', 'минута', 'включиться', 'заборский', 'получиться', 'забросить', 'быстрый', 'гол', 'задать', 'хороший', 'темп', 'поединок', 'играть', 'хороший', 'сторона', 'пять', 'очко', 'выезд', 'девять', 'хороший']


[(0, 0.8525286), (5, 0.08311545), (10, 0.041090082)]

In [31]:
x=lda.show_topics(num_topics=25, num_words=7,formatted=False)
topics_words = [(tp[0], [wd[0] for wd in tp[1]]) for tp in x]

#Below Code Prints Only Words 
for topic,words in topics_words:
    print("topic_{}: ".format(topic)+" ".join(words))

topic_0: мозг nn выяснить университет день nnn первый
topic_1: российский власть произойти тело газета россия глава
topic_2: египетский платёжный воспаление азербайджан зеландия сингапур нежели
topic_3: млн фонд nn рубль составить россия тыс
topic_4: газ египет достигать канал полоса инвестиция деловой
topic_5: ракета снижение планета фонд земля ресурс обращение
topic_6: погибнуть рак фестиваль бомба мальчик выяснить лесной
topic_7: исследование экономика земля проект экономический станция новый
topic_8: россия российский гражданин развитие рост правительство nn
topic_9: медведев озеро греция добраться туризм коллекция италия
topic_10: украина украинский пенсия район рейс расчёт nn
topic_11: рубль закон уголовный документ лицо срок завод
topic_12: москва тыс участок площадь торговый общество проект
topic_13: сша россия военный американский журнал новый научный
topic_14: писать nn день час кровь тыс препарат
topic_15: квартира дом медицина характерный лондон nthe больной
topic_16: взрыв

In [32]:
#функция векторного представления новостей

def get_lda_vector(text):
    unseen_doc = common_dictionary.doc2bow(text)
    lda_tuple = lda[unseen_doc]
    not_null_topics = dict(zip([i[0] for i in lda_tuple], [i[1] for i in lda_tuple]))

    output_vector = []
    for i in range(25):
        if i not in not_null_topics:
            output_vector.append(0)
        else:
            output_vector.append(not_null_topics[i])
    return np.array(output_vector)

In [33]:
topic_matrix = pd.DataFrame([get_lda_vector(text) for text in news['title'].values])
topic_matrix.columns = ['topic_{}'.format(i) for i in range(25)]
topic_matrix['doc_id'] = news['doc_id'].values
topic_matrix = topic_matrix[['doc_id']+['topic_{}'.format(i) for i in range(25)]]
topic_matrix.head(5)

,doc_id,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24
0,6,0.016932,0.000000,0.0,0.056480,0.0,0.000000,0.0,0.000000,0.075682,...,0.0,0.000000,0.0,0.843237,0.0,0.000000,0.0,0.0,0.0,0.000000
1,4896,0.660450,0.070643,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,...,0.0,0.245735,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000
2,4897,0.852452,0.000000,0.0,0.000000,0.0,0.083131,0.0,0.000000,0.000000,...,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000
3,4898,0.256486,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.063881,...,0.0,0.022633,0.0,0.000000,0.0,0.145708,0.0,0.0,0.0,0.486884
4,4899,0.112766,0.000000,0.0,0.296395,0.0,0.000000,0.0,0.271576,0.294508,...,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000


### Следующий шаг - векторные представления пользователей

In [34]:
users.head(3)

,uid,articles
0,u105138,"[293672, 293328, 293001, 293622, 293126, 1852]"
1,u108690,"[3405, 1739, 2972, 1158, 1599, 322665]"
2,u108339,"[1845, 2009, 2356, 1424, 2939, 323389]"


In [35]:
doc_dict = dict(zip(topic_matrix['doc_id'].values, topic_matrix[['topic_{}'.format(i) for i in range(25)]].values))

In [36]:
user_articles_list = users['articles'].iloc[33]
def get_user_embedding(user_articles_list, func):
    user_articles_list = eval(user_articles_list)
    user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
    user_vector = func(user_vector, axis=0)
    return user_vector

In [37]:
get_user_embedding(user_articles_list, np.mean)

array([0.00361711, 0.15805469, 0.        , 0.03047357, 0.00980002,
       0.0050153 , 0.00602704, 0.1042777 , 0.19651764, 0.        ,
       0.        , 0.11325389, 0.01282824, 0.18073552, 0.01724038,
       0.00866384, 0.00350384, 0.        , 0.00557389, 0.        ,
       0.02115789, 0.00943129, 0.0171107 , 0.        , 0.08233452])

###Теперь получим эмбединги для всех пользователей и проверим их качество на конкретной downstream-задаче

In [38]:
user_embeddings = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding(x, np.mean))])
user_embeddings.columns = ['topic_{}'.format(i) for i in range(25)]
user_embeddings['uid'] = users['uid'].values
user_embeddings = user_embeddings[['uid']+['topic_{}'.format(i) for i in range(25)]]
user_embeddings.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24
0,u105138,0.050865,0.097332,0.0,0.059783,0.000000,0.018414,0.004703,0.106544,0.005663,...,0.017372,0.010884,0.184219,0.015120,0.003061,0.038813,0.018065,0.003276,0.046257,0.004853
1,u108690,0.031759,0.169136,0.0,0.064190,0.002736,0.001822,0.000000,0.053580,0.072827,...,0.000000,0.002654,0.026739,0.041639,0.000000,0.039662,0.000000,0.000000,0.005282,0.165968
2,u108339,0.044625,0.189573,0.0,0.059031,0.009577,0.002159,0.005513,0.066588,0.031186,...,0.011882,0.003371,0.042364,0.045064,0.000000,0.073175,0.004660,0.016207,0.000000,0.072620


Датасет готов - можно попробовать обучить модель. Загрузим нашу разметку

In [39]:
target = pd.read_csv("users_churn.csv")
target.head(3)

,uid,churn
0,u107120,0
1,u102277,0
2,u102444,0


In [40]:
X = pd.merge(user_embeddings, target, 'left')
X.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24,churn
0,u105138,0.050865,0.097332,0.0,0.059783,0.000000,0.018414,0.004703,0.106544,0.005663,...,0.010884,0.184219,0.015120,0.003061,0.038813,0.018065,0.003276,0.046257,0.004853,0
1,u108690,0.031759,0.169136,0.0,0.064190,0.002736,0.001822,0.000000,0.053580,0.072827,...,0.002654,0.026739,0.041639,0.000000,0.039662,0.000000,0.000000,0.005282,0.165968,1
2,u108339,0.044625,0.189573,0.0,0.059031,0.009577,0.002159,0.005513,0.066588,0.031186,...,0.003371,0.042364,0.045064,0.000000,0.073175,0.004660,0.016207,0.000000,0.072620,1


In [41]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import itertools

In [42]:
#разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(X[['topic_{}'.format(i) for i in range(25)]], 
                                                    X['churn'], random_state=0)

In [43]:
logreg = LogisticRegression()
#обучим наш пайплайн
logreg.fit(X_train, y_train)

LogisticRegression()

In [44]:
#наши прогнозы для тестовой выборки
preds = logreg.predict_proba(X_test)[:, 1]
preds[:10]

array([0.22585911, 0.04880365, 0.48362854, 0.46662802, 0.06814679,
       0.08854521, 0.19416151, 0.09020489, 0.0246498 , 0.12612047])

In [45]:
from sklearn.metrics import f1_score, roc_auc_score, precision_score, classification_report, precision_recall_curve, confusion_matrix

### Рассчитаем Precision, Recall, F_score

In [46]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)
fscore = (2 * precision * recall) / (precision + recall)
ix = np.argmax(fscore)
print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds[ix], 
                                                                        fscore[ix],
                                                                        precision[ix],
                                                                        recall[ix]))

Best Threshold=0.253818, F-Score=0.684, Precision=0.616, Recall=0.767


In [47]:
roc_auc = roc_auc_score(y_test, preds)
roc_auc

0.9571347171347172

In [48]:
metrics_df = pd.DataFrame(columns=['model', 'Threshold', 'F-Score', 'Precision', 'Recall', 'Roc auc'])

metrics_df = metrics_df.append({
    'model': 'mean',
    'Threshold': thresholds[ix],
    'F-Score': fscore[ix],
    'Precision': precision[ix],
    'Recall': recall[ix],
    'Roc auc': roc_auc
}, ignore_index=True)

<ipython-input-48-bd362db29ec2>:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({


###Модифицировать код функции get_user_embedding таким образом, чтобы считалось не среднее (как в примере np.mean), а медиана.

In [49]:
user_embeddings = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding(x, np.median))])
user_embeddings.columns = ['topic_{}'.format(i) for i in range(25)]
user_embeddings['uid'] = users['uid'].values
user_embeddings = user_embeddings[['uid']+['topic_{}'.format(i) for i in range(25)]]

In [50]:
X = pd.merge(user_embeddings, target, 'left')

In [51]:
X_train, X_test, y_train, y_test = train_test_split(X[['topic_{}'.format(i) for i in range(25)]], 
                                                    X['churn'], random_state=0)
logreg.fit(X_train, y_train)
preds = logreg.predict_proba(X_test)[:, 1]

In [52]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)
fscore = (2 * precision * recall) / (precision + recall)
ix = np.argmax(fscore)
print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds[ix], 
                                                                        fscore[ix],
                                                                        precision[ix],
                                                                        recall[ix]))

roc_auc = roc_auc_score(y_test, preds)
roc_auc

Best Threshold=0.282763, F-Score=0.787, Precision=0.737, Recall=0.845


0.9743892086749228

In [53]:
metrics_df = metrics_df.append({
    'model': 'median',
    'Threshold': thresholds[ix],
    'F-Score': fscore[ix],
    'Precision': precision[ix],
    'Recall': recall[ix],
    'Roc auc': roc_auc
}, ignore_index=True)

<ipython-input-53-6778ada87dca>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({


###Повторить п.2, но используя уже не медиану, а max

In [54]:
user_embeddings = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding(x, np.max))])
user_embeddings.columns = ['topic_{}'.format(i) for i in range(25)]
user_embeddings['uid'] = users['uid'].values
user_embeddings = user_embeddings[['uid']+['topic_{}'.format(i) for i in range(25)]]

In [55]:
X = pd.merge(user_embeddings, target, 'left')

In [56]:
X_train, X_test, y_train, y_test = train_test_split(X[['topic_{}'.format(i) for i in range(25)]], 
                                                    X['churn'], random_state=0)
logreg.fit(X_train, y_train)
preds = logreg.predict_proba(X_test)[:, 1]

In [57]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)
fscore = (2 * precision * recall) / (precision + recall)
ix = np.argmax(fscore)
print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds[ix], 
                                                                        fscore[ix],
                                                                        precision[ix],
                                                                        recall[ix]))

roc_auc = roc_auc_score(y_test, preds)
roc_auc

Best Threshold=0.406001, F-Score=0.790, Precision=0.839, Recall=0.747


0.9744659573231003

In [58]:
metrics_df = metrics_df.append({
    'model': 'max',
    'Threshold': thresholds[ix],
    'F-Score': fscore[ix],
    'Precision': precision[ix],
    'Recall': recall[ix],
    'Roc auc': roc_auc
}, ignore_index=True)

<ipython-input-58-4a4bbd98b510>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({


###повторить пункт 2, но уже взвешивая новости по tfidf

In [59]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [60]:
users['articles_str'] = users['articles'].apply(lambda x: x.replace('[','').replace(']', '').replace(',', ''))

users['articles_str'].iloc[0]

'293672 293328 293001 293622 293126 1852'

In [61]:
tfidf = TfidfVectorizer()
tfidf.fit(users['articles_str'])

TfidfVectorizer()

In [62]:
idf = pd.DataFrame({'article_id': tfidf.get_feature_names_out(),
                    'idf': tfidf.idf_})

idf

,article_id,idf
0,10,8.888710
1,100,7.907880
2,1000,8.041412
3,1001,8.888710
4,1002,8.888710
...,...,...
14776,995,8.377884
14777,996,8.195562
14778,997,8.601027
14779,998,9.294175


In [63]:
def get_user_embedding_idf(user_articles_list, doc_dict):
    user_articles_list = eval(user_articles_list)
    
    user_vector = np.zeros((len(user_articles_list), 25))
    for i, doc_id in enumerate(user_articles_list):
        try:
            weight = idf[idf['article_id'] == str(doc_id)]['idf'].values[0]
        except Exception as e:
            weight = 0
        user_vector[i] = doc_dict[doc_id] * weight

    user_vector = np.mean(user_vector, axis=0)
    return user_vector

In [64]:
from tqdm import tqdm
tqdm.pandas()

user_embeddings = pd.DataFrame([i for i in users['articles'].progress_apply(lambda x: get_user_embedding_idf(x, doc_dict))])
user_embeddings.columns = [f'topic_{i}' for i in range(25)]
user_embeddings['uid'] = users['uid'].values
user_embeddings = user_embeddings[['uid']+[f'topic_{i}' for i in range(25)]]
user_embeddings.head(3)

100%|██████████| 8000/8000 [01:26<00:00, 92.95it/s] 


,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24
0,u105138,0.422456,0.839171,0.0,0.499943,0.000000,0.150917,0.040449,0.901835,0.048706,...,0.161458,0.096860,1.612439,0.123917,0.028453,0.338551,0.148056,0.030446,0.395579,0.041736
1,u108690,0.282298,1.465904,0.0,0.571214,0.024323,0.016936,0.000000,0.453083,0.589717,...,0.000000,0.022828,0.229821,0.366791,0.000000,0.349571,0.000000,0.000000,0.041147,1.436083
2,u108339,0.383576,1.593669,0.0,0.499335,0.080238,0.018090,0.047961,0.561878,0.263076,...,0.108721,0.031331,0.335298,0.378105,0.000000,0.633362,0.039341,0.126256,0.000000,0.616688


In [65]:
X = pd.merge(user_embeddings, target, 'left')

In [66]:
X_train, X_test, y_train, y_test = train_test_split(X[[f'topic_{i}' for i in range(25)]], 
                                                    X['churn'], random_state=0)
logreg.fit(X_train, y_train)
preds = logreg.predict_proba(X_test)[:, 1]

In [67]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)
fscore = (2 * precision * recall) / (precision + recall)
ix = np.argmax(fscore)
print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds[ix], 
                                                                        fscore[ix],
                                                                        precision[ix],
                                                                        recall[ix]))

roc_auc = roc_auc_score(y_test, preds)
roc_auc

Best Threshold=0.401329, F-Score=0.860, Precision=0.843, Recall=0.878


0.9877667306238734

In [68]:
metrics_df = metrics_df.append({
    'model': 'mean_idf',
    'Threshold': thresholds[ix],
    'F-Score': fscore[ix],
    'Precision': precision[ix],
    'Recall': recall[ix],
    'Roc auc': roc_auc
}, ignore_index=True)

metrics_df

<ipython-input-68-1ba231784b1c>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({


,model,Threshold,F-Score,Precision,Recall,Roc auc
0,mean,0.253818,0.683636,0.616393,0.767347,0.957135
1,median,0.282763,0.787072,0.736655,0.844898,0.974389
2,max,0.406001,0.790497,0.839450,0.746939,0.974466
3,mean_idf,0.401329,0.860000,0.843137,0.877551,0.987767


###Вывод: Метод получения эмбеддингов пользователей mean_idf оказался эффективнее остальных  в полноте и точности предсказаний, площадь под кривой ошибок максимально приближена к 1.